In [1]:
import pandas as pd
import numpy as np
import math
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from scipy.optimize import minimize
from scipy.optimize import leastsq
#from scipy.optimize import least_squares
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%run 'Features.ipynb'

In [3]:
test = pd.read_csv('input/test.csv', dtype = dtype_dict, usecols = test_cols)
test.head()

,id,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,4037,1,2209,4639078,35305
1,1,2237,1,1226,4705135,1238
2,2,2045,1,2831,4549769,32940
3,3,1227,1,4448,4717855,43066
4,4,1219,1,1130,966351,1277


In [4]:
features = feature_full_list

In [5]:
features

['12345',
 '1245o',
 '1234o',
 '124o',
 '1235o',
 '125o',
 '123o',
 '12o',
 '2345o 1o',
 '2345o',
 '245o',
 '234o',
 '24o',
 '235o',
 '25o',
 '1345o',
 '145o',
 '134o',
 '14o',
 '23o',
 '135o',
 '15o',
 '13o',
 '1o',
 '2o',
 '345o',
 '45o',
 '34o',
 '35o',
 '4o',
 '5o',
 '3o']

In [6]:
features_trained = {'2345o 1o': [-3.5731738918328126, 1.0046309257624915, 0.90862462801484511, 73.610153038782173]}

In [7]:
features_trained['2345o 1o'][:-1]

[-3.5731738918328126, 1.0046309257624915, 0.9086246280148451]

In [8]:
for feature in features:
    part = merge_new_feature(test, feature, folder = output_folder)
    
    sub_features = feature.split()
    
    index = pd.isnull(part[sub_features[0]])
    for i in range(1, len(sub_features)):
        index = index | pd.isnull(part[sub_features[i]])
    test = part[index]
    
    part = part[~index]
    
    if feature in features_trained.keys():
        w = features_trained[feature][:-1]
        part[target] = w[0]*np.ones(len(part))
        for i in range(0, len(sub_features)):
            part[target] = part[target] + w[i+1]*part[sub_features[i]]
    else:
        part[target] = part[sub_features[0]]
        
    index = (part[target]<0)
    test_ = part[index]
    test_.drop(target, axis=1, inplace=True)  
    
    test = pd.concat([test, test_])
    
    for i in range(0,len(sub_features)):
        del test[sub_features[i]]

    part = part[~index]    
    #part[target] = np.exp(part[target])
        
    print feature, ' : ', len(test) + len(part), len(test), len(part), part[target].mean(), part[target].median()
    
    if feature == features[0]:
        part.to_csv('output/my_submission.csv', columns = [u'id', u'Demanda_uni_equil'])
    else:    
        part.to_csv('output/my_submission.csv', mode = 'a', header=False,  columns = [u'id', u'Demanda_uni_equil'])
           

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


12345  :  6999251 3973680 3025571 10.1197322533 4.66666666667
1245o  :  3973680 1446137 2527543 5.21679275938 3.0
1234o  :  1446137 1446024 113 102.15037104 82.6732142857
124o  :  1446024 1446024 0 nan nan
1235o  :  1446024 1445846 178 27.9122092526 1.29166666666
125o  :  1445846 1445846 0 nan nan
123o  :  1445846 1445804 42 69.7767007031 62.6576403695
12o  :  1445804 1445802 2 5.875 5.875
2345o 1o  :  1445802 475862 969940 4.77534590668 3.24139817563
2345o  :  475862 380528 95334 3.5084168993 1.71631664293
245o  :  380528 380493 35 6.30576015255 2.5
234o  :  380493 175583 204910 5.49126647896 3.35433272947
24o  :  175583 175047 536 15.1325932029 8.15801886793
235o  :  175047 160046 15001 10.5267923135 4.03438438438
25o  :  160046 159317 729 12.5348236474 6.57142857143
1345o  :  159317 4788 154529 4.52082171835 3.64635854342
145o  :  4788 4787 1 40.9507936508 40.9507936508
134o  :  4787 4787 0 nan nan
14o  :  4787 4787 0 nan nan
23o  :  4787 1997 2790 5.88048149294 2.14285714286
135o  

In [9]:
my_submission = pd.read_csv('output/my_submission.csv', usecols = ['id', 'Demanda_uni_equil'])
#my_submission[target] = np.rint(my_submission[target]).astype(int)
#my_submission[target] = np.rint(np.exp(my_submission[target])).astype(int)

#my_submission.loc[my_submission[target].isnull(), target] = 

my_submission = my_submission.sort_values('id').reset_index(drop = True)

print len(my_submission)
my_submission.head()

6999251


,id,Demanda_uni_equil
0,0,4.000000
1,1,0.348566
2,2,2.000000
3,3,0.474706
4,4,5.671429


In [10]:
my_submission[target].describe()

count    6.999251e+06
mean     7.259815e+00
std      1.988699e+01
min      0.000000e+00
25%      2.000000e+00
50%      3.666667e+00
75%      7.000000e+00
max      4.800000e+03
Name: Demanda_uni_equil, dtype: float64

In [11]:
len(my_submission[my_submission[target].isnull()])

0

In [12]:
my_submission.to_csv('output/my_submission.csv', index=False, columns=['id','Demanda_uni_equil'])

In [13]:
my_submission = pd.read_csv('output/my_submission.csv')
my_submission.head()

,id,Demanda_uni_equil
0,0,4.000000
1,1,0.348566
2,2,2.000000
3,3,0.474706
4,4,5.671429
